In [ ]:
import ROOT as rt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def getReweightSF(path, infile, outfile) :
    
    inputFilePath = path + "/" + infile
    infile=rt.TFile.Open(inputFilePath, 'READ')
    outfile = rt.TFile.Open(path + "/" + outfile, "RECREATE")
    
    unfoldInputHist_PT_MASS = infile.Get("folded/Pt/histo_UnfoldInput")
    projectedReco_PT_MASS = infile.Get("folded/Pt/histo_ProjectedReco")
    unfoldInputHist_PT_MASS.Divide(projectedReco_PT_MASS) 
    unfoldInputHist_PT_MASS.SetName("reweightSF_PT_MASS")
    
    unfoldInputHist_MASS = infile.Get("folded/Mass/histo_UnfoldInput")
    projectedReco_MASS = infile.Get("folded/Mass/histo_ProjectedReco")
    unfoldInputHist_MASS.Divide(projectedReco_MASS) 
    unfoldInputHist_MASS.SetName("reweightSF_MASS")
    
    outfile.cd()
    unfoldInputHist_PT_MASS.Write()
    unfoldInputHist_MASS.Write()
    
    outfile.Close()
    infile.Close()
    

In [ ]:
year = "2018"
channel = "muon"

getReweightSF("/home/jhkim/ISR_Run2/unfolding/TUnfoldISR2016/output/" + year + "/" + channel + "_detector_dressedDRp1_extended",
              "DetUNFOLD_" + channel + "_" + year + ".root", "reweightSF.root")

In [ ]:
matrix_file = rt.TFile.Open("/home/jhkim/ISR_Run2/unfolding/TUnfoldISR2016/inFiles/2017/electron_detector_dressedDRp1/matrix/DY.root", 'READ')
matrix = matrix_file.Get("Detector_Dressed_DRp1_Fiducial/Pt_ResMatrix_FineCoarse/hmcPtGenRec")

In [ ]:
projectedReco = matrix.ProjectionY("histo_ProjectedReco", 1, -1, "e")

c = rt.TCanvas("canvas","The Canvas Title",800,600)
projectedReco.Draw()
c.Draw()


In [ ]:
reweight_file = rt.TFile.Open("/home/jhkim/ISR_Run2/unfolding/TUnfoldISR2016/output/2017/electron_detector_dressedDRp1/reweightSF.root")
reweight = reweight_file.Get("reweightSF_PT_MASS")



In [ ]:

c = rt.TCanvas("reweight","The Canvas Title",800,600)
reweight.Draw()
c.Draw()



In [ ]:
matrix_reweighted = matrix.Clone("matrix_reweighted")

for ybin in range(matrix.GetNbinsY()) : 
    
    sf = reweight.GetBinContent(ybin)
    for xbin in range(matrix.GetNbinsX()) :
        temp_content = matrix.GetBinContent(xbin, ybin)
        matrix_reweighted.SetBinContent(xbin, ybin, temp_content*sf)
        

In [ ]:
unfold_file = rt.TFile.Open("/home/jhkim/ISR_Run2/unfolding/TUnfoldISR2016/output/2017/electron_detector_dressedDRp1/DetUNFOLD_electron_2017.root", "READ")
unfold_input = unfold_file.Get("folded/Pt/histo_UnfoldInput")

In [ ]:
projectedReco_reweighted = matrix_reweighted.ProjectionY("histo_ProjectedReco", 1, -1, "e")

In [ ]:
c = rt.TCanvas("canvas_reweighted","The Canvas Title",800,600)
c.SetLogy()
projectedReco.Draw()
projectedReco_reweighted.Draw("Same")
projectedReco_reweighted.SetLineColor(rt.kRed)
print(projectedReco.Integral(), projectedReco_reweighted.Integral())
unfold_input.SetMarkerStyle(11)
unfold_input.Draw("SAME P")
c.Draw()


In [ ]:
def readdir(file, outfile, inpath = "", outpath = "", new_hist_postfix = "", target_postfix = "", new_target_postfix = None, only_hist = True) :
    print("readir called....")
    dirSave = file.GetDirectory(inpath)
    keys = dirSave.GetListOfKeys() 
    
    for key in keys :
        if key.IsFolder() :
            print("folder: " , key.GetName())
            if inpath == "" :
                out_dir = key.GetName()
                
                if new_hist_postfix != "" and new_hist_postfix in out_dir :
                    temp_list = out_dir.split("_")
                    temp_list.remove(new_hist_postfix)
                    out_dir = "_".join(temp_list) 
                
                print("creat a directory... ", out_dir)
                outfile.mkdir(out_dir)
                readdir(file, outfile, 
                        key.GetName() + "/", out_dir + "/", 
                        new_hist_postfix, target_postfix, new_target_postfix, only_hist)
            else :
                
                out_dir = key.GetName()
                
                if new_hist_postfix != "" and new_hist_postfix in out_dir :
                    out_dir = "_".join(out_dir.split("_")[:-1]) #
                    
                print("creat a directory... ", outpath + out_dir + "/")
                outfile.mkdir(outpath + out_dir + "/")
                readdir(file, outfile, 
                        inpath + key.GetName() + "/", outpath + out_dir + "/", 
                        new_hist_postfix, target_postfix, new_target_postfix, only_hist)
            continue
            
        temp_obj = file.Get(inpath + "/" + key.GetName()) 
        
        if type(temp_obj) == rt.TH1D or type(temp_obj) == rt.TH2D:
            if new_hist_postfix != "" :
                print("Histogram ", key.GetName(), type(temp_obj))
                temp_obj.SetName(key.GetName() + "_" + new_hist_postfix)
                outfile.cd(outpath)
                temp_obj.Write()
            else :
                if target_postfix in key.GetName() :
                    outfile.cd(outpath)
                    if new_target_postfix != None :
                        temp_obj.SetName("_".join(key.GetName().split("_")[:-1]))
                    temp_obj.Write()
        else :
            if only_hist == False :
                outfile.cd(outpath)
                temp_obj.Write()
    

In [ ]:
# For lepton momentum correction

channel = "electron"


inputHistFilePath = "./inFiles/2016/" + channel + "_detector_dressedDRp1_extended_v2/input/unfold_input_raw_NoLepMomCorr"
file=rt.TFile.Open(inputHistFilePath + ".root", 'READ')
ofile = rt.TFile.Open(inputHistFilePath + "_new.root", "RECREATE")
#readdir(file, ofile, "", "", dir1 + dir2)
readdir(file, ofile, "", "", "", "NoLepMomCorr","", False)
ofile.Close()



In [ ]:
# For lepton momentum correction
prefix_list = ["LepMomScale", "LepMomRes"]
postfix_list = ["Up", "Down"]
channel = "electron"
year = "2016"

for dir1 in prefix_list :
    for dir2 in postfix_list :
        inputHistFilePath = "./inFiles/" + year + "/" + channel + "_detector_preFSR/input/unfold_input" + dir1 + dir2
        file=rt.TFile.Open(inputHistFilePath + ".root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "_new.root", "RECREATE")
        #readdir(file, ofile, "", "", dir1 + dir2)
        readdir(file, ofile, "", "", "")
        ofile.Close()

In [ ]:
# For lepton momentum correction
prefix_list = ["LepMomScale", "LepMomRes"]
postfix_list = ["Up", "Down"]

for dir1 in prefix_list :
    for dir2 in postfix_list :
        inputHistFilePath = "./inFiles/" + year + "/" + channel + "_detector_preFSR/matrix/unfold_matrix_v2" + dir1 + dir2
        file=rt.TFile.Open(inputHistFilePath + ".root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "_new.root", "RECREATE")
        readdir(file, ofile, "", "", "")
        ofile.Close()

In [ ]:
channel = "muon"
year = "2018"

inputHistFilePath = "./inFiles/" + year + "/" + channel + "_detector_preFSR/input/"
file = rt.TFile.Open(inputHistFilePath + "unfold_input_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "unfold_input_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()

In [ ]:
channel = "muon"
year = "2018"

inputHistFilePath = "./inFiles/" + year + "/" + channel + "_detector_preFSR/matrix/"
file = rt.TFile.Open(inputHistFilePath + "DY_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "DY_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()


In [ ]:

inputHistFilePath = "./inFiles/" + year + "/" + channel + "_detector_dressedDRp1_extended/matrix/"
file = rt.TFile.Open(inputHistFilePath + "DY_FSR_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "DY_FSR_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()


In [ ]:

inputHistFilePath = "./inFiles/"+ year + "/" + channel + "_detector_preFSR/efficiency/"
file =  rt.TFile.Open(inputHistFilePath + "acceptance_dRp1_pdf.root", "READ")
ofile = rt.TFile.Open(inputHistFilePath + "acceptance_dRp1_pdf_new.root", "RECREATE")

readdir(file, ofile, "", "", "", "PDF")
ofile.Close()

In [ ]:
# For lepton momentum correction
prefix_list = ["LepMomScale"]
postfix_list = ["Up", "Down"]
year = "2017"

for dir1 in prefix_list :
    for dir2 in postfix_list :
        inputHistFilePath = "./inFiles/" + year + "/muon_detector_preFSR/input/unfold_input" + dir1 + dir2
        file=rt.TFile.Open(inputHistFilePath + ".root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "_new.root", "RECREATE")
        #readdir(file, ofile, "", "", dir1 + dir2)
        readdir(file, ofile, "", "", "")
        ofile.Close()

In [ ]:
# For lepton momentum correction
prefix_list = ["LepMomScale"]
postfix_list = ["Up", "Down"]

for dir1 in prefix_list :
    for dir2 in postfix_list :
        inputHistFilePath = "./inFiles/" + year + "/muon_detector_preFSR/matrix/unfold_matrix_v2" + dir1 + dir2
        file=rt.TFile.Open(inputHistFilePath + ".root", 'READ')
        ofile = rt.TFile.Open(inputHistFilePath + "_new.root", "RECREATE")
        readdir(file, ofile, "", "", "")
        ofile.Close()